# Social Media Mining - Hackathon 

## Predicting the Popularity of Online Content
Replicate and extend the article (link below) for predicting online content popularity using Twitter hashtags. Applying a method to a different dataset often entails making different choices from the ones of the original authors. Describe your experience and motivate your unique approach.

### Dataset
Using the Twitter Streaming API, you will collect a dataset of Twitter hashtags.

### References
- [Link to article](http://www.hpl.hp.com/research/scl/papers/predictions/predictions.pdf)
- [Twitter Streaming API](https://developer.twitter.com/en/docs/tweets/sample-realtime/overview/GET_statuse_sample)


In [ ]:
import twitter
import json
import os

CONSUMER_KEY = 'n3klbsDYyvvk1IxCEvm7W3tGw'
CONSUMER_SECRET ='CJZ71dbUlhjDu7DyzawOCBMPkltdpw6816WZ3WUgC8rbOcKSs8'
OAUTH_TOKEN = '1031697780417417216-TwltGeXHoG0eybrj8tF1AAckCj2bDb'
OAUTH_TOKEN_SECRET = 'skAVmJwY1BhbrPWAeJB5uOWMacpikpYw2n8XAXnUI3TwU'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)
tStream = twitter.TwitterStream(auth = auth)
tSample = tStream.statuses.sample() # stream sample

### NameFile() creates a file name to make sure we don't overwite previous sample

In [ ]:
def NameFile():
    pathName = os.path.join('Data', 'Tweets')
    
    if not os.path.isfile(pathName + '.json'):
        return pathName + '.json'
    
    count = 1
    while (os.path.isfile(pathName + str(count) + '.json')):
        count += 1
            
    return pathName + str(count) + '.json'

### StoreTweets() writes sampled tweets to ../Data/Tweets.json
Filters out deleted tweets and tweets without hashtags

In [ ]:
def StoreTweets(pathToTweets, tSample):
    with open(os.path.join(pathToTweets), 'w') as f:
        f.write('[')

        count = 0
        for tweet in tSample:
            #if (list(tweet.keys())[0] != 'delete') and (tweet.get('entities', False)):
            if (tweet.get('delete', False) is False) and (tweet.get('entities', False)):
                if tweet['entities'].get('hashtags', False):
                    count += 1
                    json.dump(tweet, f)
                    if count < 100: f.write(',\n')
                    else: break
        f.write(']')
        f.close()
        print(count, "tweets written to file")
    return

In [ ]:
#pathToTweets = NameFile()
#StoreTweets(pathToTweets, tSample)  

### LoadFile() opens the specified file and returns a json object of its contents


In [ ]:
def LoadFile(pathName):    
    if os.path.isfile(pathName):
        file = open(os.path.join(pathName), 'r')
        return json.load(file)
    else: print("File not in directory")

In [ ]:
tweets = LoadFile(os.path.join('Data', 'Tweets.json'))

### StoreHashtags() extracts all unique hashtags from LoadTweets() and writes them to Tweets(#)-Hashtags.json

In [ ]:
def StoreHashtags(pathName, givenTweets):
    hashtagList = []
    for tweet in givenTweets:
        for hTag in tweet['entities']['hashtags']:
            hashtagList.append(hTag['text'])
        
    hashtagList = list(set(hashtagList))
    newPath = pathName[:-5] + "-Hashtags.json"

    with open(newPath, 'w') as f:    
        json.dump(hashtagList, f)
        f.close()
    print("Stored all unique hashtags in {}".format(newPath))

In [ ]:
#StoreHashtags(os.path.join('Data', 'Tweets.json'), tweets)

In [ ]:
hashtagList = LoadFile(os.path.join('Data', 'Tweets-Hashtags.json'))
#print(json.dumps(hashtags, indent = 1))

In [ ]:
for i, tag in enumerate(hashtagList):
    print(tag)
    if i > 5: break

### StoreDict() takes in the list of hashtag strings (hashtagList), constructs a dictionary of hashtag dictionaries, writes them to Tweets(#)-Hashtags-Dict.json

In [ ]:
def StoreDictionary(pathName, givenHTList, numberOfDays):
    # use of other hashtags, media(type?), user_mentions, filter_level
    details = {'occurrences': 0, 'created_at': [], 'followers_count': [], 'lang': {}}
    
    days = {}    
    for i in range(numberOfDays):
        days.setdefault('day' + str(i), dict(details))

    hashtagsDict = {}
    for i, tag in enumerate(givenHTList):
        hashtagsDict.setdefault(tag, dict(days))
    
    newPath = pathName[:-5] + "-Dict.json"
    with open(newPath, 'w') as f:    
        json.dump(hashtagsDict, f)
        f.close()
    print("Stored the hashtag dictionary in {}".format(newPath))
    return 

In [ ]:
# StoreDictionary(os.path.join('Data', 'Tweets-Hashtags_test.json'), hashtagList, 30)

In [ ]:
hashtags = LoadFile(os.path.join('Data', 'Tweets-Hashtags-Dict.json'))

In [ ]:
print("Unique hashtags:", len(hashtags), end = '\n\n')
for tag, content in hashtags.items():
    print("Hashtag:", tag)
    print(json.dumps(content, indent = 1))
    break

### Hashtag Dictionary: 
- **keys:** hashtag string
- **values:** dictionary of day objects

### day objects:
- **occurences:** number of occurences of hashtag for given day
- **created_at:** list (strings) of creation times of all (or some?) tweets using hashtag
- **followers_count:** list (ints) of number followers the user has who posted the tweet
- **lang:** dictionary to count the number of self-declared languages by user
    - "en": 100
    - "es": 50

### Added by Logan
Search function demonstrated below.
Run to see example with #turkey from Oct 12 to Oct 14 -- takes a minute to run
Also can use search.search_by_hashtag() to search one range only

In [1]:
#search function is under ./Code/search.py
import sys
sys.path.append("./Code/") #add the path with the code so that it can be imported

import search

#searches for #turkey from oct 12 to oct 14 then prints debug stats
#function returns a list of dicts with the tweets and information
#read the comments in search.py for more information

results = search.searchOverRange("#turkey", 2018, 10, 12, 2018, 10, 14, debug=1)


day: 12, number of tweets: 1184
day: 13, number of tweets: 94


In [4]:
#prints the twitter metadata
print(results[1]["metadata"])

{'cache-control': 'no-cache, no-store, must-revalidate, pre-check=0, post-check=0', 'content-disposition': 'attachment; filename=json.json', 'content-encoding': 'gzip', 'content-length': '56822', 'content-type': 'application/json;charset=utf-8', 'date': 'Thu, 18 Oct 2018 17:12:49 GMT', 'expires': 'Tue, 31 Mar 1981 05:00:00 GMT', 'last-modified': 'Thu, 18 Oct 2018 17:12:49 GMT', 'pragma': 'no-cache', 'server': 'tsa_b', 'set-cookie': 'personalization_id="v1_smyOoxKH425CVXX0Dxl3ig=="; Expires=Sat, 17 Oct 2020 17:12:49 GMT; Path=/; Domain=.twitter.com, lang=en; Path=/, guest_id=v1%3A153988276971541185; Expires=Sat, 17 Oct 2020 17:12:49 GMT; Path=/; Domain=.twitter.com', 'status': '200 OK', 'strict-transport-security': 'max-age=631138519', 'x-access-level': 'read-write', 'x-connection-hash': 'f7464f70a9db12ab4a5343f5606120da', 'x-content-type-options': 'nosniff', 'x-frame-options': 'SAMEORIGIN', 'x-rate-limit-limit': '180', 'x-rate-limit-remaining': '128', 'x-rate-limit-reset': '1539882995'